In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

class ClothingRecommendationModel:
    def __init__(self, input_shape=(224, 224, 3), embedding_dim=256):
        """
        Initialize the clothing recommendation model
        
        Args:
            input_shape (tuple): Input image dimensions
            embedding_dim (int): Dimension of the image embedding vector
        """
        self.input_shape = input_shape
        self.embedding_dim = embedding_dim
        self.model = self._build_model()
    
    def _build_model(self):
        """
        Build the recommendation model architecture
        
        Returns:
            tf.keras.Model: Compiled recommendation model
        """
        # Use pre-trained ResNet50 as base model
        base_model = ResNet50(
            weights='imagenet', 
            include_top=False, 
            input_shape=self.input_shape
        )
        
        # Freeze base model layers
        for layer in base_model.layers:
            layer.trainable = False
        
        # Custom embedding layers
        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        
        # Final embedding layer
        embedding = layers.Dense(
            self.embedding_dim, 
            activation='relu', 
            name='embedding_layer'
        )(x)
        
        # Create model
        model = models.Model(
            inputs=base_model.input, 
            outputs=embedding
        )
        
        # Compile model
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
            loss='cosine_similarity'
        )
        
        return model
    
    def train(self, train_data, epochs=10, batch_size=32):
        """
        Train the recommendation model
        
        Args:
            train_data (tf.data.Dataset): Training dataset
            epochs (int): Number of training epochs
            batch_size (int): Batch size for training
        """
        # Data augmentation
        data_augmentation = tf.keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.2),
            layers.RandomZoom(0.2),
        ])
        
        # Train the model
        history = self.model.fit(
            train_data,
            epochs=epochs,
            batch_size=batch_size
        )
        
        return history
    
    def get_similar_images(self, reference_image, image_database, top_k=5):
        """
        Find similar images based on embedding similarity
        
        Args:
            reference_image (np.array): Input reference image
            image_database (list): List of images to compare
            top_k (int): Number of similar images to return
        
        Returns:
            list: Indices of top similar images
        """
        # Get embedding for reference image
        reference_embedding = self.model.predict(
            np.expand_dims(reference_image, axis=0)
        )
        
        # Compute embeddings for database
        database_embeddings = self.model.predict(image_database)
        
        # Compute cosine similarity
        similarities = tf.keras.losses.cosine_similarity(
            reference_embedding, 
            database_embeddings
        ).numpy()
        
        # Find top-k most similar images
        top_indices = np.argsort(similarities)[0][:top_k]
        
        return top_indices

# Example usage
def prepare_dataset(image_directory):
    """
    Prepare image dataset for training
    
    Args:
        image_directory (str): Path to clothing image directory
    
    Returns:
        tf.data.Dataset: Prepared training dataset
    """
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )
    
    train_generator = datagen.flow_from_directory(
        image_directory,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )
    
    return train_generator

def save_model(self, save_path='clothing_recommendation_model'):
    """
    Save the trained model to local file
    
    Args:
        save_path (str): Directory to save model
    """
    # Create directory if it doesn't exist
    os.makedirs(save_path, exist_ok=True)
    
    # Save full model
    full_model_path = os.path.join(save_path, 'full_model')
    self.model.save(full_model_path)
    
    # Save weights separately
    weights_path = os.path.join(save_path, 'model_weights.weights.h5')
    self.model.save_weights(weights_path)
    
    print(f"Model saved to {save_path}")

@classmethod
def load_model(cls, load_path='clothing_recommendation_model'):
    """
    Load a previously saved model
    
    Args:
        load_path (str): Directory containing saved model
    
    Returns:
        ClothingRecommendationModel: Loaded model instance
    """
    # Create model instance
    model_instance = cls()
    
    # Try to load full model first
    full_model_path = os.path.join(load_path, 'full_model')
    weights_path = os.path.join(load_path, 'model_weights.weights.h5')
    
    try:
        # Attempt to load full model
        model_instance.model = tf.keras.models.load_model(full_model_path)
        print("Full model loaded successfully.")
    except:
        # If full model loading fails, rebuild and load weights
        model_instance.model = model_instance._build_model()
        model_instance.model.load_weights(weights_path)
        print("Model weights loaded successfully.")
    
    return model_instance

# Main execution example
def main():
    # Initialize model
    recommendation_model = ClothingRecommendationModel()
    
    # Prepare dataset
    train_data = prepare_dataset('path/to/clothing/images')
    
    # Train model
    recommendation_model.train(train_data)

    # Save the model
    recommendation_model.save_model('my_clothing_model')

    # Load the model (demonstrates loading functionality)
    loaded_model = ClothingRecommendationModel.load_model('my_clothing_model')
    
    print("Clothing Recommendation Model Training Complete!")

if __name__ == '__main__':
    main()

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import os

class ClothingRecommendationModel:
    def __init__(self, input_shape=(224, 224, 3), embedding_dim=256):
        """Initialize clothing recommendation model"""
        self.input_shape = input_shape
        self.embedding_dim = embedding_dim
        self.model = self._build_model()
    
    def _build_model(self):
        """Build recommendation model architecture"""
        # Base model with ResNet50
        base_model = ResNet50(
            weights='imagenet', 
            include_top=False, 
            input_shape=self.input_shape
        )
        
        # Freeze base model layers
        for layer in base_model.layers:
            layer.trainable = False
        
        # Custom embedding layers with ReLU
        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        
        # Final embedding layer
        embedding = layers.Dense(
            self.embedding_dim, 
            activation='relu', 
            name='embedding_layer'
        )(x)
        
        # Create and compile model
        model = models.Model(
            inputs=base_model.input, 
            outputs=embedding
        )
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
            loss='cosine_similarity'
        )
        
        return model
    
    def load_and_preprocess_image(self, image_path):
        """Load and preprocess single image"""
        try:
            # Load image
            img = load_img(image_path, target_size=self.input_shape)
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array /= 255.0
            return img_array
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            return None
    
    def prepare_dataset(self, csv_path):
        """Prepare dataset from CSV"""
        # Read CSV
        df = pd.read_csv(csv_path)

        print("CSV Loaded: ", df.head()) 
        
        # Filter valid images
        df = df[df['local_path'].apply(os.path.exists)]
        
        # Prepare images and categories
        images = []
        categories = []
        
        for path, category in zip(df['local_path'], df['category']):
            img = self.load_and_preprocess_image(path)
            if img is not None:
                images.append(img[0])
                categories.append(category) 
        
        return np.array(images), np.array(categories)
    
    def train(self, csv_path, epochs=10, batch_size=32):
        """Train recommendation model"""
        # Prepare dataset
        X, y = self.prepare_dataset(csv_path)
        
        print(X, y)

        # Split data
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # Train model
        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size
        )
        
        return history
    
    def find_similar_images(self, reference_image_path, image_database_df, top_k=5):
        """Find similar images based on embedding similarity"""
        # Preprocess reference image
        ref_img = self.load_and_preprocess_image(reference_image_path)
        
        if ref_img is None:
            return []
        
        # Get reference embedding
        ref_embedding = self.model.predict(ref_img)
        
        # Prepare database embeddings
        db_images = []
        db_paths = []
        
        for path in image_database_df['local_path']:
            img = self.load_and_preprocess_image(path)
            if img is not None:
                db_images.append(img[0])
                db_paths.append(path)
        
        # Convert to numpy array
        db_images = np.array(db_images)
        
        # Compute database embeddings
        db_embeddings = self.model.predict(db_images)
        
        # Compute similarities
        similarities = tf.keras.losses.cosine_similarity(
            ref_embedding, 
            db_embeddings
        ).numpy()
        
        # Find top-k similar images
        top_indices = np.argsort(similarities[0])[:top_k]
        
        return [db_paths[i] for i in top_indices]
    
    def save_model(self, save_path='clothing_recommendation_model'):
        """Save trained model"""
        os.makedirs(save_path, exist_ok=True)
        self.model.save(os.path.join(save_path, 'full_model'))
        print(f"Model saved to {save_path}")
    
    @classmethod
    def load_model(cls, load_path='clothing_recommendation_model'):
        """Load saved model"""
        model_instance = cls()
        model_instance.model = tf.keras.models.load_model(
            os.path.join(load_path, 'full_model')
        )
        return model_instance

# Example usage
def main():
    # Initialize model
    rec_model = ClothingRecommendationModel()
    
    # Train model
    rec_model.train('../Dataset/final_dataset_id_local_paths.csv')
    
    # Save model
    rec_model.save_model('clothing_recommendation')
    
    # Load model (demonstration)
    loaded_model = ClothingRecommendationModel.load_model('clothing_recommendation')
    
    # Find similar images
    similar_images = loaded_model.find_similar_images(
        'saree.jpeg', 
        pd.read_csv('clothing_dataset.csv')
    )
    print("Similar Images:", similar_images)

if __name__ == '__main__':
    main()

CSV Loaded:     Unnamed: 0.2  id   category  \
0             0   0  shapewear   
1             1   1    tshirts   
2             2   2       tops   
3             3   3       tops   
4             4   4    tshirts   

                                           image_url  Unnamed: 0.1  \
0  https://assets.myntassets.com/h_720,q_90,w_540...           NaN   
1  https://assets.myntassets.com/h_720,q_90,w_540...           NaN   
2  https://assets.myntassets.com/h_720,q_90,w_540...           NaN   
3  https://assets.myntassets.com/h_720,q_90,w_540...           NaN   
4  https://assets.myntassets.com/h_720,q_90,w_540...           NaN   

   Unnamed: 0              local_path  
0         NaN  processed_images/0.jpg  
1         NaN  processed_images/1.jpg  
2         NaN  processed_images/2.jpg  
3         NaN  processed_images/3.jpg  
4         NaN  processed_images/4.jpg  
[[[[0.9411765  0.9254902  0.88235295]
   [0.9411765  0.9254902  0.88235295]
   [0.9411765  0.9254902  0.88235295]
   ...


KeyboardInterrupt: 

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import os

# Configure GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

class ClothingRecommendationModel:
    def __init__(self, input_shape=(224, 224, 3), embedding_dim=256):
        """Initialize clothing recommendation model"""
        self.input_shape = input_shape
        self.embedding_dim = embedding_dim
        self.label_encoder = None  # For label encoding
        self.model = self._build_model()

    def _build_model(self):
        """Build recommendation model architecture"""
        # Base model with ResNet50
        base_model = ResNet50(
            weights="imagenet",
            include_top=False,
            input_shape=self.input_shape
        )

        # Freeze base model layers
        for layer in base_model.layers:
            layer.trainable = False

        # Custom embedding layers with ReLU
        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(512, activation="relu")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)

        # Final embedding layer
        embedding = layers.Dense(
            self.embedding_dim,
            activation="relu",
            name="embedding_layer"
        )(x)

        # Create and compile model
        model = models.Model(
            inputs=base_model.input,
            outputs=embedding
        )
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
            loss="cosine_similarity"
        )

        return model

    def load_and_preprocess_image(self, image_path):
        """Load and preprocess single image"""
        try:
            # Load image
            img = load_img(image_path, target_size=self.input_shape[:2])
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array /= 255.0
            return img_array
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            return None

    def prepare_dataset(self, csv_path):
        """Prepare dataset from CSV"""
        # Read CSV
        df = pd.read_csv(csv_path)

        # Filter valid images
        df = df[df["local_path"].apply(os.path.exists)]

        # Prepare images and categories
        images = []
        categories = []

        for path, category in tqdm(
            zip(df["local_path"], df["category"]),
            total=len(df),
            desc="Preparing dataset"
        ):
            img = self.load_and_preprocess_image(path)
            if img is not None:
                images.append(img[0])  # Add image shape correction
                categories.append(category)

        # Convert to numpy array
        images = np.array(images)

        # Encode categories into numeric labels
        self.label_encoder = LabelEncoder()
        categories = self.label_encoder.fit_transform(categories)  # Numeric encoding

        return images, categories

    # def train(self, csv_path, epochs=10, batch_size=16):
    #     """Train recommendation model"""
    #     # Prepare dataset
    #     X, y = self.prepare_dataset(csv_path)

    #     # Split data
    #     X_train, X_val, y_train, y_val = train_test_split(
    #         X, y, test_size=0.2, random_state=42
    #     )

    #     # Use categorical crossentropy instead of cosine similarity
    #     self.model.compile(
    #         optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    #         loss='sparse_categorical_crossentropy',
    #         metrics=['accuracy']
    #     )

    #     print("Training model:")
    #     # Train model
    #     history = self.model.fit(
    #         X_train, y_train,
    #         validation_data=(X_val, y_val),
    #         epochs=epochs,
    #         batch_size=batch_size,
    #         verbose=1  # Use Keras's built-in progress bar
    #     )

    #     return history

    def train(self, csv_path, epochs=10, batch_size=16):
        """Train recommendation model"""
        # Prepare dataset
        X, y = self.prepare_dataset(csv_path)
        
        # Split data
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # Compile model
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        print("Training model:")
        try:
            # Train model with early stopping
            early_stopping = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss', 
                patience=3, 
                restore_best_weights=True
            )
            
            history = self.model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=epochs,
                batch_size=batch_size,
                callbacks=[early_stopping],
                verbose=1
            )
            
            return history
        except Exception as e:
            print(f"Training interrupted: {e}")
            return None
        
    def find_similar_images(self, reference_image_path, image_database_df, top_k=5):
        """Find similar images based on embedding similarity"""
        # Preprocess reference image
        ref_img = self.load_and_preprocess_image(reference_image_path)

        if ref_img is None:
            return []

        # Get reference embedding
        ref_embedding = self.model.predict(ref_img)

        # Prepare database embeddings
        db_images = []
        db_paths = []

        for path in tqdm(
            image_database_df["local_path"],
            desc="Processing database images"
        ):
            img = self.load_and_preprocess_image(path)
            if img is not None:
                db_images.append(img[0])
                db_paths.append(path)

        # Convert to numpy array
        db_images = np.array(db_images)

        # Compute database embeddings
        db_embeddings = self.model.predict(db_images)

        # Compute similarities
        ref_embedding = np.expand_dims(ref_embedding, axis=0)  # Ensure proper shape
        similarities = tf.keras.losses.cosine_similarity(
            ref_embedding,
            db_embeddings
        ).numpy()

        # Find top-k similar images
        top_indices = np.argsort(similarities[0])[:top_k]

        return [db_paths[i] for i in top_indices]

    def save_model(self, save_path="clothing_recommendation_model"):
        """Save trained model"""
        os.makedirs(save_path, exist_ok=True)
        # Add full filepath with .keras extension
        full_filepath = os.path.join(save_path, "full_model.keras")
        self.model.save("full_model.keras")
        print(f"Model saved to {full_filepath}")

    @classmethod
    def load_model(cls, load_path="clothing_recommendation_model"):
        """Load saved model"""
        model_instance = cls()
        model_instance.model = tf.keras.models.load_model(
            os.path.join(load_path, "full_model.keras")
        )
        return model_instance


# Example usage
def main():
    # Initialize model
    rec_model = ClothingRecommendationModel()

    # Train model
    # rec_model.train("../Dataset/final_dataset_id_local_paths.csv")

    # Save model
    rec_model.save_model("clothing_recommendation")

    # Load model (demonstration)
    loaded_model = ClothingRecommendationModel.load_model("clothing_recommendation")

    # Find similar images
    similar_images = loaded_model.find_similar_images(
        "saree.jpeg",
        pd.read_csv("../Dataset/final_dataset_id_local_paths.csv")
    )
    print("Similar Images:", similar_images)


if __name__ == "__main__":
    main()


Model saved to clothing_recommendation\full_model.keras


ValueError: File not found: filepath=clothing_recommendation\full_model.keras. Please ensure the file is an accessible `.keras` zip file.

In [6]:
rec_model = ClothingRecommendationModel()

def save_model(self, save_path="clothing_recommendation_model"):
        """Save trained model"""
        os.makedirs(save_path, exist_ok=True)
        # Add full filepath with .keras extension
        full_filepath = os.path.join(save_path, "full_model.keras")
        self.model.save("full_model.keras")
        print(f"Model saved to {full_filepath}")

rec_model.save_model("clothing_recommendation")

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=clothing_recommendation\full_model.

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import os

# Load the model
loaded_model = tf.keras.models.load_model(os.path.join("full_model.keras"))

# Create a new ClothingRecommendationModel instance
rec_model = ClothingRecommendationModel()
rec_model.model = loaded_model

# Now use the loaded model for recommendations
similar_images = rec_model.find_similar_images(
    'saree.jpeg', 
    pd.read_csv("../Dataset/final_dataset_id_local_paths.csv")
)

d:\Python\envs\ml\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


NameError: name 'ClothingRecommendationModel' is not defined

In [10]:
print("Similar Images:", similar_images)

Similar Images: ['processed_images/4899.jpg', 'processed_images/8185.jpg', 'processed_images/8064.jpg', 'processed_images/5863.jpg', 'processed_images/986.jpg']


In [2]:
print(X, y)

NameError: name 'X' is not defined

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import os


In [ ]:
class ClothingRecommendationModel:
    def __init__(self, input_shape=(224, 224, 3), embedding_dim=256):
        """Initialize clothing recommendation model"""
        self.input_shape = input_shape
        self.embedding_dim = embedding_dim
        self.model = self._build_model()
    
    def _build_model(self):
        """Build recommendation model architecture"""
        # Base model with ResNet50
        base_model = ResNet50(
            weights='imagenet', 
            include_top=False, 
            input_shape=self.input_shape
        )
        
        # Freeze base model layers
        for layer in base_model.layers:
            layer.trainable = False
        
        # Custom embedding layers with ReLU
        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        
        # Final embedding layer
        embedding = layers.Dense(
            self.embedding_dim, 
            activation='relu', 
            name='embedding_layer'
        )(x)
        
        # Create and compile model
        model = models.Model(
            inputs=base_model.input, 
            outputs=embedding
        )
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
            loss='cosine_similarity'
        )
        
        return model


In [ ]:
def load_and_preprocess_image(self, image_path):
    """Load and preprocess single image"""
    try:
        # Load image
        img = load_img(image_path, target_size=self.input_shape)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0
        return img_array
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None


In [ ]:
def prepare_dataset(self, csv_path):
    """Prepare dataset from CSV"""
    # Read CSV
    df = pd.read_csv(csv_path)

    print("CSV Loaded: ", df.head()) 
    
    # Filter valid images
    df = df[df['local_path'].apply(os.path.exists)]

    print("df in preprare dataset",df)
    
    # Prepare images and categories
    images = []
    categories = []
    
    for path, category in zip(df['local_path'], df['category']):
        img = self.load_and_preprocess_image(path)
        if img is not None:
            images.append(img[0])
            categories.append(category) 
    
    return np.array(images), np.array(categories)


In [ ]:
def train(self, csv_path, epochs=10, batch_size=32):
    """Train recommendation model"""
    # Prepare dataset
    X, y = self.prepare_dataset(csv_path)
    
    # Split data
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Train model
    history = self.model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size
    )
    
    return history


In [ ]:
def find_similar_images(self, reference_image_path, image_database_df, top_k=5):
    """Find similar images based on embedding similarity"""
    # Preprocess reference image
    ref_img = self.load_and_preprocess_image(reference_image_path)
    
    if ref_img is None:
        return []
    
    # Get reference embedding
    ref_embedding = self.model.predict(ref_img)
    
    # Prepare database embeddings
    db_images = []
    db_paths = []
    
    for path in image_database_df['local_path']:
        img = self.load_and_preprocess_image(path)
        if img is not None:
            db_images.append(img[0])
            db_paths.append(path)
    
    # Convert to numpy array
    db_images = np.array(db_images)
    
    # Compute database embeddings
    db_embeddings = self.model.predict(db_images)
    
    # Compute similarities
    similarities = tf.keras.losses.cosine_similarity(
        ref_embedding, 
        db_embeddings
    ).numpy()
    
    # Find top-k similar images
    top_indices = np.argsort(similarities[0])[:top_k]
    
    return [db_paths[i] for i in top_indices]


In [ ]:
def save_model(self, save_path='clothing_recommendation_model'):
    """Save trained model"""
    os.makedirs(save_path, exist_ok=True)
    self.model.save(os.path.join(save_path, 'full_model'))
    print(f"Model saved to {save_path}")

@classmethod
def load_model(cls, load_path='clothing_recommendation_model'):
    """Load saved model"""
    model_instance = cls()
    model_instance.model = tf.keras.models.load_model(
        os.path.join(load_path, 'full_model')
    )
    return model_instance


In [ ]:
# Example usage
def main():
    # Initialize model
    rec_model = ClothingRecommendationModel()
    
    # Train model
    rec_model.train('../Dataset/final_dataset_id_local_paths.csv')
    
    # Save model
    rec_model.save_model('clothing_recommendation')
    
    # Load model (demonstration)
    loaded_model = ClothingRecommendationModel.load_model('clothing_recommendation')
    
    # Find similar images
    similar_images = loaded_model.find_similar_images(
        'saree.jpeg', 
        pd.read_csv('clothing_dataset.csv')
    )
    print("Similar Images:", similar_images)

if __name__ == '__main__':
    main()


In [ ]:
def train(self, csv_path, epochs=10, batch_size=16):
        # Prepare dataset
        X, y = self.prepare_dataset(csv_path)

        # Create data generator
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True
        )

        # Split data
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        # Recompile model with appropriate loss function
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
            loss='sparse_categorical_crossentropy',  # Change loss function
            metrics=['accuracy']
        )

        # Train model
        history = self.model.fit(                                                                                  
            datagen.flow(X_train, y_train, batch_size=batch_size),
            validation_data=(X_val, y_val),
            epochs=epochs,
            steps_per_epoch=len(X_train) // batch_size,
            verbose=1
        )

        return history